In [19]:
# This code is only used to check the correctness of B-X3DH
import numpy as np
import sys
from numpy.polynomial import polynomial as pol
import random
from math import floor
import hashlib
from Crypto.Cipher import AES
import os
from Crypto.Util import Counter
import base64
import secrets
def AES_key_gen():
    K = secrets.token_bytes(32) # a 32 bytes key
    return K
def AES_enc(K: bytes, M: str) -> str:
    # K: AES key (16/24/32 bytes), M: plaintext string
    nonce = secrets.token_bytes(8)  # 64-bit nonce
    ctr = Counter.new(64, prefix=nonce, initial_value=0)
    cipher = AES.new(K, AES.MODE_CTR, counter=ctr)
    ciphertext = cipher.encrypt(M)
    return base64.b64encode(nonce + ciphertext).decode('utf-8')
def AES_dec(K: bytes, C: str) -> str:
    data = base64.b64decode(C)
    nonce = data[:8]
    ciphertext = data[8:]
    ctr = Counter.new(64, prefix=nonce, initial_value=0)
    cipher = AES.new(K, AES.MODE_CTR, counter=ctr)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext
def shake256(seed: bytes, outlen: int) -> bytes:
    return hashlib.shake_256(seed).digest(outlen)

def bit_count(x):
    return bin(x).count("1")
def cbd_eta(input_bytes: bytes, eta=2, n: int = 256):
    """
    CBD_eta
    """
    assert len(input_bytes) == eta * n // 4
    coeffs = [0] * n
    b_int = int.from_bytes(input_bytes, "little")
    mask = (1 << eta) - 1
    mask2 = (1 << 2 * eta) - 1
    for i in range(n):
        x = b_int & mask2
        a = bit_count(x & mask)
        b = bit_count((x >> eta) & mask)
        coeffs[i] = (a - b)
        b_int >>= 2 * eta
    return np.array(coeffs)

def prg_gaussian(seed: bytes, n: int):
    input_bytes = shake256(seed, 2 * n // 4)
    return cbd_eta(input_bytes, 2, n)

def generate_s_b_gaussian(seed: bytes,a, n, q):
    assert len(seed) == 32  # 256-bit
    s = prg_gaussian(seed + b"s", n)
    e = prg_gaussian(seed + b"e", n)
    # b = a * s +2e mod q
    b = pol.polymul(a, s)%q
    e= pol.polymul(2,e)%q
    b = pol.polyadd(b,e) % q
    b=np.round(pol.polydiv(b,quotient(n))[1])%q
    return s, b
def generate_s_b_gaussian1(seed: bytes,a, n, q):
    assert len(seed) == 32  # 256-bit
    s = prg_gaussian(seed + b"s", n)
    # b = a * s mod q
    b = pol.polymul(a, s)%q
    b=np.round(pol.polydiv(b,quotient(n))[1])%q
    return s, b
#function Q=x^n +1
def quotient(n):
    Q=xN_1 = [1] + [0] * (n-1) + [1]
    return Q

def derrive_a(n,q):
        #x=np.random.randint(-(q-1)/2,(q-1)/2+1,n)
    x=[]
    for i in range(n):
        x.append(np.random.randint(0,q))
    return x
def mod2(x,w,q):
    x=((x+w*(q-1)/2)%q)%2
    return x
def sig_0(x,q):
    if x>= -(floor(q/4)) and x<=(floor(q/4)):
        y=0
    else:
        y=1
    return y
def sig_1(x,q):
    if x>= (-floor(q/4)+1) and x<=(floor(q/4)+1):
        y=0
    else:
        y=1
    return y
def keygen(a,n,q):
    seed=os.urandom(32)
    return generate_s_b_gaussian(seed,a, n, q)

def astRLDH(pk_a,sk_b,q,n):
    e = prg_gaussian(os.urandom(32), n)
    # b = a * s +2e mod q
    b = pol.polymul(pk_a, sk_b)%q
    kb=np.round(pol.polydiv(b,quotient(n))[1])%q
    for t in range(n):
        if kb[t]>(q-1)/2:
            kb[t]=kb[t]-(q-1)
            if kb[t]==0:
                kb[t]=kb[t]+1
    htb=[]
    dhb=[]
    for t in range(n):
        alea=random.randint(0,1)
        if alea==1:
            htb.append(sig_1(kb[t],q))
        else:
            htb.append(sig_0(kb[t],q))
        dhb.append(floor(mod2(kb[t],htb[t],q)))
    return htb,dhb
def recRLDH(pk_b,sk_a,htb,q,n):
    ka=np.round(pol.polydiv(pol.polymul(pk_b,sk_a),quotient(n))[1])%q
    for t in range(n):
        if ka[t]>(q-1)/2:
            ka[t]=ka[t]-(q-1)
            if ka[t]==0:
                ka[t]=ka[t]+1
    dha=[]
    for t in range(n):
        dha.append(floor(mod2(ka[t],htb[t],q)))
    return dha
def KDF(dh1_bytes: bytes, sid: bytes, out_len=32) -> tuple:
    data = dh1_bytes + sid
    k = hashlib.shake_256(data+b'k').digest(out_len)
    k0 = hashlib.shake_256(data+b'k0').digest(out_len)
    k1 = hashlib.shake_256(data+b'k1').digest(out_len)
    return k, k0, k1
def B_X3DH(n,q):
    # Key exchange
    # 0. Alice key gen
    sid=b"Alice" + b"Bob"
    dh1_bytes=b''
    dh2_bytes=b''
    a=derrive_a(n,q)
    lsk_a,lpk_a=keygen(a,n,q)
    seed=os.urandom(32)
    seeda_blind=hashlib.shake_256(seed+b'ablind').digest(32)
    seeda_eph=hashlib.shake_256(seed+b'aeph').digest(32)
    bsk_a,bpk_a=generate_s_b_gaussian1(seeda_blind,a, n, q)
    esk_a,epk_a=generate_s_b_gaussian1(seeda_eph,a, n, q)
    blinded_lpk_a=pol.polyadd(lpk_a,bpk_a)%q
    # 0. Bob key gen
    lsk_b,lpk_b=keygen(a,n,q)
    seed1=os.urandom(32)
    seedb_blind=hashlib.shake_256(seed1+b'bblind').digest(32)
    seedb_eph=hashlib.shake_256(seed1+b'beph').digest(32)
    bsk_b,bpk_b=generate_s_b_gaussian1(seedb_blind,a, n, q)
    esk_b,epk_b=generate_s_b_gaussian1(seedb_eph,a, n, q)
    blinded_lpk_b=pol.polyadd(lpk_b,bpk_b)%q
    # 1. Alice->Bob: blinded_lpk_a,epk_a 
    sum_pk_a=pol.polyadd(blinded_lpk_a,epk_a)%q
    sum_sk_b=pol.polyadd(lsk_b,bsk_b)%q
    sum_sk_b=pol.polyadd(sum_sk_b,esk_b)%q
    # 2. Bob->Alice: blinded_lpk_b,epk_b,htb_1,htb_2 
    htb_1,dhb_1=astRLDH(sum_pk_a,sum_sk_b,q,n)
    # 3. Alice: Calculate shared secret
    sum_pk_b=pol.polyadd(blinded_lpk_b,epk_b)%q
    sum_sk_a=pol.polyadd(lsk_a,bsk_a)%q
    sum_sk_a=pol.polyadd(sum_sk_a,esk_a)%q
    dha_1=recRLDH(sum_pk_b,sum_sk_a,htb_1,q,n)
#     Test keys
    if dhb_1 == dha_1:
        sid = sid + sum_pk_a.tobytes() + sum_pk_b.tobytes() + bytes(htb_1)
        print(True)
    else:
        print(False)
        return False
    # session key generation
    dh1_bytes=dh1_bytes+bytes(dhb_1)
    k_shared, k0, k1 = KDF(dh1_bytes, sid)
    # Authentication
    # 1. Alice->Bob: Enc(k0,(seed,Hash(lpk_a)))
    h=shake256(lpk_a.tobytes(),32)
    m_a=seed+h
    c_0=AES_enc(k0,m_a) # Beside c_0, Alice can send other message using k
    # 2. Bob: recover lpk_a and verify
    # 2.1 recover and verify Alice
    m_a_s=AES_dec(k0,c_0) # m_a_s is byte-type
    seed_rec = m_a_s[:32]
    h_rec = m_a_s[32:]

    seeda_blind=hashlib.shake_256(seed_rec+b'ablind').digest(32)
    seeda_eph=hashlib.shake_256(seed_rec+b'aeph').digest(32)
    bsk_a,bpk_a=generate_s_b_gaussian1(seeda_blind,a, n, q)
    esk_a,epk_a=generate_s_b_gaussian1(seeda_eph,a, n, q)
    
    lpk_aa=pol.polysub(sum_pk_a,bpk_a)%q
    lpk_aa=pol.polysub(lpk_aa,epk_a)%q
    if h_rec==shake256(lpk_aa.tobytes(),32):
        print(True)
    # 2.2 Bob->Alice: Enc(k1(seed1,Enc(k_e,Hash(lkp_b))))
    htb_2,dhb_2=astRLDH(epk_a,esk_b,q,n)
    sid = sid + sum_pk_a.tobytes() + sum_pk_b.tobytes()+epk_a.tobytes()+epk_b.tobytes() + bytes(htb_1)+ bytes(htb_2)
    dh2_bytes=bytes(dhb_2)
    k_shared1, k00, _ = KDF(dh2_bytes, sid)
    h=shake256(lpk_b.tobytes(),32)
    c_t=AES_enc(k00,h)
    m_b=seed1+bytes(htb_2)
    c_1=AES_enc(k1,m_b)
    # 3. Alcie: recover lpk_a and verify
    m_b_s=AES_dec(k1,c_1) # m_a_s is byte-type
    seed_rec = m_b_s[:32]
    hta_2 = list(m_b_s[32:])
    seedb_blind=hashlib.shake_256(seed_rec+b'bblind').digest(32)
    seedb_eph=hashlib.shake_256(seed_rec+b'beph').digest(32)
    bsk_b,bpk_b=generate_s_b_gaussian1(seedb_blind,a, n, q)
    esk_b,epk_b=generate_s_b_gaussian1(seedb_eph,a, n, q)
    
    lpk_bb=pol.polysub(sum_pk_b,bpk_b)%q
    lpk_bb=pol.polysub(lpk_bb,epk_b)%q
    dha_2=recRLDH(epk_b,esk_a,hta_2,q,n)
    k_shared1, k00, _ = KDF(bytes(dha_2), sid)
    h_rec=AES_dec(k00,c_t)
    if h_rec==shake256(lpk_bb.tobytes(),32):
        print(True)
    return int.from_bytes(k_shared1, byteorder='big')^ int.from_bytes(k_shared, byteorder='big')
def test():
    key=B_X3DH(n=768,q=15631) #
    print("shared:=",key.to_bytes(32,byteorder='big'))
    key=B_X3DH(n=1280,q=25201) #
    print("shared:=",key.to_bytes(32,byteorder='big'))
test()

True
True
True
shared:= b't\x96\x1a/\xc17\xa9@\xd3\xefM\x0b_\xc0j\xa52\xe9\x17V\x8c\n\xa3\xf2\x01\n\xea\xf6\x8ck\xc7\x1f'
True
True
True
shared:= b'O\xb1J\xe9nW\xc6L\xd7CcoZ\xa7\xc7\xf0\xfd\xed\x01Y\xdb\x1aFt\x93\x91\xf9 \xd1\xec\x8c\x01'


In [ ]:
# Use sage to execute this estimator
from estimator.lwe_parameters import LWEParameters as LWEest
from estimator import *

params = LWEest(
     n=768,
     q=15631,#768,2.2,128 bit
     Xs=ND.CenteredBinomial(2),
     Xe=ND.CenteredBinomial(2),
     m=256,
     tag="example"
)
r=LWE.estimate(params)

params = LWEest(
     n=1280,
     q=25201,#1280,256 bit
     Xs=ND.CenteredBinomial(2),
     Xe=ND.CenteredBinomial(2),
     m=256,
     tag="example"
)
r=LWE.estimate(params)

In [ ]:
"""
For (768,15631,2), best attack is dual_hybrid with rop: ≈2^168.0, failure rate is 2^-128

bkw                  :: rop: ≈2^225.2, m: ≈2^212.4, mem: ≈2^213.3, b: 15, t1: 0, t2: 20, ℓ: 14, #cod: 678, #top: 0, #test: 92, tag: coded-bkw
usvp                 :: rop: ≈2^215.5, red: ≈2^215.5, δ: 1.002767, β: 666, d: 1024, tag: usvp
bdd                  :: rop: ≈2^209.8, red: ≈2^208.3, svp: ≈2^209.2, β: 640, η: 674, d: 1025, tag: bdd
dual                 :: rop: ≈2^229.2, mem: ≈2^153.4, m: 256, β: 712, d: 1024, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^168.0, red: ≈2^167.7, guess: ≈2^165.5, β: 486, p: 4, ζ: 20, t: 50, β': 486, N: ≈2^99.6, m: 768

For (1280,25201,2), best attack is dual_hybrid with rop: ≈2^271.0, failure rate is 2^-256

bkw                  :: rop: ≈2^353.7, m: ≈2^339.8, mem: ≈2^340.8, b: 23, t1: 0, t2: 23, ℓ: 22, #cod: 1139, #top: 1, #test: 140, tag: coded-bkw
usvp                 :: rop: ≈2^411.7, red: ≈2^411.7, δ: 1.001605, β: 1370, d: 1536, tag: usvp
dual                 :: rop: ≈2^445.8, mem: ≈2^306.2, m: 256, β: 1495, d: 1536, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^271.1, red: ≈2^271.0, guess: ≈2^266.5, β: 853, p: 4, ζ: 20, t: 100, β': 835, N: ≈2^172.8, m: 1280

"""